<a href="https://colab.research.google.com/github/YuriVSA/DataScience-Estudos/blob/main/DataScience_Preprocessamento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução
Objetivo desse notebook é dar seguimento nos estudos de DataScience, resumindo a parte de Preprocessamento estudada.


Integração com o google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
DRIVE_DIRECTORY = "Curso_DataScience"
DRIVE_DIRECTORY = os.path.join("/content/drive/MyDrive", DRIVE_DIRECTORY)
if not os.path.exists(DRIVE_DIRECTORY):
  os.makedirs(DRIVE_DIRECTORY)

### Importação bibliotecas

In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import numpy as np

### Preprocessamento das bases

As anotações e detalhes do preprocessamento estão em [link github](https://github.com/YuriVSA/DataScience-Estudos/blob/main/DataScience_Python.ipynb)

#### Base de crédito


Essa base é uma adaptação do site kaggle, e tem esse seguinte formato:


clientid - variável categorica nominal;

income - variável numerica contínua;

age - variável contínua;

loan - variável contínua;

default - variável categória nominal binária;

##### Tratamento

In [ ]:
base_credit = pd.read_csv("credit_data.csv")

In [ ]:
base_credit[base_credit["age"] < 0] #Vamos resolver essa situação

,clientid,income,age,loan,default
15,16,50501.726689,-28.218361,3977.287432,0
21,22,32197.620701,-52.423280,4244.057136,0
26,27,63287.038908,-36.496976,9595.286289,0


In [ ]:
base_credit[base_credit["age"] > 0].mean()

,0
clientid,1003.431795
income,45328.856915
age,40.927700
loan,4443.240892
default,0.141926


In [ ]:
base_credit.loc[base_credit["age"] < 0, "age"] = 40.92

In [ ]:
base_credit[base_credit["age"] < 0]

,clientid,income,age,loan,default


In [ ]:
base_credit.iloc[[15, 21, 26]]

,clientid,income,age,loan,default
15,16,50501.726689,40.92,3977.287432,0
21,22,32197.620701,40.92,4244.057136,0
26,27,63287.038908,40.92,9595.286289,0


In [ ]:
base_credit.isnull().sum() #Verifica a quantidade de null's da base

,0
clientid,0
income,0
age,3
loan,0
default,0


In [ ]:
base_credit.loc[base_credit["age"].isnull()]

,clientid,income,age,loan,default
28,29,59417.805406,NaN,2082.625938,0
30,31,48528.852796,NaN,6155.784670,0
31,32,23526.302555,NaN,2862.010139,0


In [ ]:
base_credit["age"].fillna(40.92, inplace=True)

In [ ]:
base_credit.loc[base_credit["age"].isnull()]


,clientid,income,age,loan,default


##### Divisão classe e previsores


In [ ]:
lista_previsores = ["income", "age", "loan"]

X_credit = base_credit[lista_previsores].values

In [ ]:
lista_classe = ["default"]
Y_credit = base_credit[lista_classe].values.ravel()

##### Split Treino e Teste

In [ ]:
X_credit_train, X_credit_test, Y_credit_train, Y_credit_test = train_test_split(X_credit, Y_credit, test_size=0.25, random_state=0)

In [ ]:
X_credit_train.shape

(1500, 3)

In [ ]:
X_credit_test.shape

(500, 3)

In [ ]:
Y_credit_test.shape, Y_credit_train.shape

((500,), (1500,))

##### Escalonamento

A padronização é feita com os dados de média e desvio padrão apenas da base separada de treino, para evitar vazamento de dados.

In [ ]:
escalonamento = StandardScaler()
X_credit_train = escalonamento.fit_transform(X_credit_train)
X_credit_test = escalonamento.transform(X_credit_test)

In [ ]:
X_credit_train[0]

array([-1.38977008,  0.51327562,  0.09505576])

#### Base census



Nessa base de dados temos:

**age:** va. numerica discreta;

**workclass:** va. categorica nominal;

**final-weight:** va. numerica continua; (faixa de valores não é bem definida, ou finita)

**education:** va. categorica ordinal;

**education-num:** va. numerica discreta;

**marital-status, occupation, relationship, race, sex:** va. categorica nominal

**capital_gain, capital_loss:** va. continua; (faixa de valores nao bem definida)

**hour-per-week:** va. discreta;

**native-country:** va. nominal;

**income:** va. nominal binomial ou ordinal;


In [ ]:
base_census = pd.read_csv("census.csv")

##### Divisão entre previsores e classe


In [ ]:
X_census = base_census.iloc[:, 0:14].values

In [ ]:
X_census[0]

array([39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married',
       ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40,
       ' United-States'], dtype=object)

In [ ]:
Y_census = base_census.iloc[:, 14].values.ravel()

In [ ]:
Y_census.shape

(32561,)

##### Split Treino e Teste


In [ ]:
X_census_train, X_census_test, Y_census_train, Y_census_test = train_test_split(X_census, Y_census, test_size=0.15, random_state=0)

##### Categorização e Escalonamento

In [ ]:
#Definição do preprocessor com padronização e onehotencoder
lista_one_hot_encoder = [1, 3, 5, 6, 7, 8, 9, 13]
lista_padronizacao = [0, 2, 4, 10, 11, 12]

preprocessor = ColumnTransformer(
    transformers = [
        ("one_hot", OneHotEncoder(sparse_output=True, handle_unknown='ignore'), lista_one_hot_encoder),
        ("padronizacao", StandardScaler(), lista_padronizacao)
    ],
    remainder = "passthrough"
)
#handle_unknown evita erros caso, no futuro, tenha alguma informação nova em relação aos dados categoricos
#ele tratará como desconhecido

In [ ]:
#Padronização com base no treinamento
X_census_train = preprocessor.fit_transform(X_census_train)
X_census_test = preprocessor.transform(X_census_test)

In [ ]:
print(X_census_train[0])


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14 stored elements and shape (1, 108)>
  Coords	Values
  (0, 6)	1.0
  (0, 20)	1.0
  (0, 27)	1.0
  (0, 36)	1.0
  (0, 47)	1.0
  (0, 57)	1.0
  (0, 59)	1.0
  (0, 99)	1.0
  (0, 102)	-0.9215007779032642
  (0, 103)	1.849772888104273
  (0, 104)	-0.4167533886395352
  (0, 105)	-0.14635426916744584
  (0, 106)	-0.21593706966925938
  (0, 107)	-0.036804296872061014


In [ ]:
X_census_train.shape, Y_census_train.shape

((27676, 108), (27676,))

In [ ]:
X_census_test.shape, Y_census_test.shape

((4885, 108), (4885,))

#### Salvando as variáveis

In [ ]:
import pickle

In [ ]:
with open(os.path.join(DRIVE_DIRECTORY, 'credit.pkl'), mode='wb') as f:
  pickle.dump([X_credit_train, Y_credit_train, X_credit_test, Y_credit_test], f)

In [ ]:
with open(os.path.join(DRIVE_DIRECTORY, 'census.pkl'), mode='wb') as f:
  pickle.dump([X_census_train, Y_census_train, X_census_test, Y_census_test], f)